In [0]:
from torch.utils.data import Dataset, DataLoader
from torch import nn, from_numpy, optim
import numpy as np
#pandas- librărie pentru lucrul cu fișierele
import pandas as pd
import torch
import torch.nn as nn

In [0]:
df=pd.read_csv("/content/winequality-red.csv",header=None)

In [81]:
df.values

array([[ 7.4  ,  0.7  ,  0.   , ...,  0.56 ,  9.4  ,  0.   ],
       [ 7.8  ,  0.88 ,  0.   , ...,  0.68 ,  9.8  ,  0.   ],
       [ 7.8  ,  0.76 ,  0.04 , ...,  0.65 ,  9.8  ,  0.   ],
       ...,
       [ 6.3  ,  0.51 ,  0.13 , ...,  0.75 , 11.   ,  1.   ],
       [ 5.9  ,  0.645,  0.12 , ...,  0.71 , 10.2  ,  0.   ],
       [ 6.   ,  0.31 ,  0.47 , ...,  0.66 , 11.   ,  1.   ]])

In [0]:
#Dataset - o clasă din PyTorch foarte utilă gestionării seturilor de date
class WineDataset(Dataset):
    """ Diabetes dataset."""
    # Initialize your data, download, etc.
    def __init__(self):
        #Citim setul de date
        df=pd.read_csv("/content/winequality-red.csv",header=None, dtype=np.float32)
        xy = torch.from_numpy(df.values)
        self.len = xy.shape[0]
        #Vom folosi ca input toate valorile mai puțin ultima coloană
        self.x_data = xy[:, 0:-1]
        #Vom folosi ca output ultima coloană
        self.y_data = xy[:, [-1]]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [0]:
dataset = WineDataset()
#DataLoader - un utilitar ce ne ajută să împărțim setul de date pe batch-uri și astfel să facem antrenare în mod Mini-Batch
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=1)

In [84]:
dataset[0]

(tensor([ 7.4000,  0.7000,  0.0000,  1.9000,  0.0760, 11.0000, 34.0000,  0.9978,
          3.5100,  0.5600,  9.4000]), tensor([0.]))

In [0]:
class Model(nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = nn.Linear(11, 100)
        self.l2 = nn.Linear(100, 50)
        self.l3 = nn.Linear(50, 30)
        self.l4 = nn.Linear(30, 80)
        self.l5 = nn.Linear(80, 10)
        self.l6 = nn.Linear(10, 1)

        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.relu(self.l1(x))
        out2 = self.relu(self.l2(out1))
        out3 = self.relu(self.l3(out2))
        out4 = self.relu(self.l4(out3))
        out5 = self.relu(self.l5(out4))
        y_pred = self.sigmoid(self.l6(out5))
        return y_pred

In [0]:
model=Model()

In [0]:
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [0]:
# Training loop
for epoch in range(100):
  #Aplicăm modificarea de learning rate per epocă, deci va trebui să avem un istoric al loss-ului
  for i, data in enumerate(train_loader, 0):
      # get the inputs
      inputs, labels = data

      # Forward pass: Compute predicted y by passing x to the model
      y_pred = model(inputs)

      # Compute and print loss
      loss = criterion(y_pred, labels)
      print(f'Epoch {epoch + 1} | Batch: {i+1} | Loss: {loss.item():.4f}')

      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()